### Loading in and organzing 

In [214]:
import pandas as pd

# List from https://www.satdump.org/Satellite-List/ file save as txt

inputBandMask = 'X-Band'
file1 = r"SatDump\Satellite List SatDump.txt"
with open(file1,'r', encoding="utf8") as fyl:
    text = fyl.read()
    
mySatellites = []
myFrequencies = []
frequencyHolder = []
NoradID = []
allFreq = []
nextLinesFreq = False
with open(file1, 'r', encoding="utf8") as topo_file:
    for line in topo_file:
        if "Norad" in line:
            mySatellites.append(line.split(" [")[0].lstrip())
            noradString = line.split(" [")[1].lstrip()
            noradString = noradString.split(" ")[1]
            newNorad = []
            for var in noradString:
                if var.isnumeric():
                    newNorad.append(var)
                else:
                    break
            singleElementNORAD = ''.join(newNorad)
            NoradID.append(singleElementNORAD)
        if nextLinesFreq == True:
            band = line.split(" ")[0]
            frequencyHolder.append(band)
            allFreq.append(band)
        if "Frequency" in line:
            nextLinesFreq = True
        if len(frequencyHolder) > 0:
            if len(line.strip()) == 0:
                if "\n" in frequencyHolder: frequencyHolder.remove('\n')
                singleElement = ', '.join(frequencyHolder)
                myFrequencies.append(singleElement)
                frequencyHolder = []
                nextLinesFreq = False
myDF = pd.DataFrame(
    {'Satellite': mySatellites,
     'Frequency Band': myFrequencies,
     'Norad ID': NoradID
    })

UniqueFreq = list(set(allFreq))
satsInDesiredBand = myDF[myDF['Frequency Band'].str.contains(inputBandMask)==True]["Satellite"].tolist()
print("The Frequency Bands are:\n\n", UniqueFreq)
print("\nAll satellites in", inputBandMask, 'are:\n\n', satsInDesiredBand)

The Frequency Bands are:

 ['RTD', 'MDL', 'and', 'HRD', 'X-band', 'HRPT', 'LRIT', 'stations,', 'GRB', 'GVAR', 'C-Band', 'AHRPT', 'S-band', 'S-VISSR', 'Ku-Band', 'CDA', 'APT', 'Hartebeesthoek', 'LRPT', 'GAC', 'X-Band', 'L-band', 'SD', 'or', 'DPT', 'xRIT', 'HRIT', '\n', 'GSM', 'Ka-Band', 'L-Band', 'VHF', 'CDAS', 'RDAS', 'DSB', 'MPT', 'S-Band']

All satellites in X-Band are:

 ['METEOR-M N1', 'METEOR-M N2', 'METEOR-M N2-2', 'METEOR-M N2-3', 'METEOR-M N2-4', 'METOP-A', 'METOP-B', 'METOP-C', 'PROBA-V', 'PROBA-V cubesat companion', 'CORIOLIS', 'SMAP', 'TUBIN', 'IRIS', 'OPS-SAT', 'HINODE', 'SWOT', 'FORMOSAT-5', 'SUOMI-NPP', 'SMOS', 'AQUA', 'TERRA', 'AURA', 'CRYOSAT-2', 'GCOM-W1', 'GCOM-C1', 'CALIPSO', 'PAZ', 'CFOSAT', 'PRISMA', 'AEOLUS', 'STEREO A/B', 'LANDSAT 8/9', 'SENTINEL 6', 'GOES-16', 'GOES-17', 'GOES-18']


### making TLE

In [281]:
with open('active.txt','r', encoding="utf8") as tle_text_read:
    tle_text = tle_text_read.read()
    
f = open(f"TLE_files/{inputBandMask}_TLE_satDump.txt", "w")
notFound = []
Found = []
FoundNORAD = []
for DesSat in satsInDesiredBand:
    if DesSat in tle_text:
        myIndex = tle_text.index(DesSat)
        myRange = tle_text[myIndex:myIndex+200]
        myRangeSplit = myRange.split('\n')
        print(myRangeSplit[0], file=f)
        print(myRangeSplit[1], file=f)
        print(myRangeSplit[2], file=f)
        Found.append(DesSat)
    else:
        myNewNorad = myDF.loc[myDF['Satellite'] == DesSat, 'Norad ID'].iloc[0]
        myNewNoradCheck = "1 "+myNewNorad+"U"
        myNewNoradCheckreverse = myNewNoradCheck[::-1]
        
        if myNewNoradCheck in tle_text:
            myIndex = tle_text.index(myNewNoradCheck)
            myRange = tle_text[myIndex-5:myIndex+200]
            myRangeSplit = myRange.split('\n')
            FoundNORAD.append(DesSat)
            getSatNameRange = tle_text[myIndex-200:myIndex+200]
            getSatNameRangeReverse = getSatNameRange[::-1]
            print(getSatNameRangeReverse[192::].split("\n")[1][::-1].strip(), file=f)
            print(myRangeSplit[1], file=f)
            print(myRangeSplit[2], file=f)
        else:
            notFound.append(DesSat)
        
f.close()   
print(f"TLE Output saved to: TLE_files/{inputBandMask}_TLE_satDump.txt")
print()
print("We found the TLE with no name changes for:\n", Found)
print("\nWe found the TLE based on NORAD, with name changes for:\n", FoundNORAD)
print("\nWe did NOT find the TLE for:\n", notFound)

TLE Output saved to: X-Band_TLE_satDump.txt

We found the TLE with no name changes for:
 ['METOP-B', 'METOP-C', 'PROBA-V', 'CORIOLIS', 'SMAP', 'TUBIN', 'IRIS', 'OPS-SAT', 'HINODE', 'SWOT', 'FORMOSAT-5', 'SMOS', 'AQUA', 'TERRA', 'AURA', 'GCOM-W1', 'CALIPSO', 'PAZ', 'CFOSAT', 'PRISMA']

We found the TLE based on NORAD, with name changes for:
 ['METEOR-M N2', 'METEOR-M N2-2', 'METEOR-M N2-3', 'METEOR-M N2-4', 'PROBA-V cubesat companion', 'SUOMI-NPP', 'CRYOSAT-2', 'GCOM-C1', 'LANDSAT 8/9', 'SENTINEL 6', 'GOES-16', 'GOES-17', 'GOES-18']

We did NOT find the TLE for:
 ['METEOR-M N1', 'METOP-A', 'AEOLUS', 'STEREO A/B']
